In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df_head = pd.read_csv('input/ecommerce-behavior/2019-Nov.csv', nrows=5)
df_head

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [25]:
usecols = ['event_time', 'event_type', 'product_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']

categorical_dtypes = {
    'event_type':'category', 'product_id':'category',
    'category_code':'category', 'brand':'category', 
    'user_id':'category', 'user_session':'category'
}

In [26]:
df = pd.read_csv('input/ecommerce-behavior/2019-Nov.csv', dtype=categorical_dtypes, usecols=usecols)
print(df.shape)
print(df.columns)
print(df.dtypes)
df.tail()

(67501979, 8)
Index(['event_time', 'event_type', 'product_id', 'category_code', 'brand',
       'price', 'user_id', 'user_session'],
      dtype='object')
event_time         object
event_type       category
product_id       category
category_code    category
brand            category
price             float64
user_id          category
user_session     category
dtype: object


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
67501974,2019-11-30 23:59:58 UTC,view,15700137,NaN,NaN,277.74,532714000,02b4131c-0112-4231-aafa-ceaa08e77c1b
67501975,2019-11-30 23:59:58 UTC,view,28719425,apparel.shoes,baden,62.81,545223467,734c5eef-0742-4f8b-9d22-48f75b0bc359
67501976,2019-11-30 23:59:59 UTC,view,1004833,electronics.smartphone,samsung,167.03,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb
67501977,2019-11-30 23:59:59 UTC,view,2701706,appliances.kitchen.refrigerators,samsung,566.27,531607492,368ddc8b-5db9-40fb-b7ff-b6582a1192c0
67501978,2019-11-30 23:59:59 UTC,view,1004233,electronics.smartphone,apple,1312.52,579969851,90aca71c-ed8a-4670-866a-761ebacb732d


In [28]:
print(df["event_type"].sample(frac=0.01).nunique())
df["event_type"].value_counts()

3


view        63556110
cart         3028930
purchase      916939
Name: event_type, dtype: int64

In [29]:
df['purchase'] = np.where(df["event_type"]=="purchase", 1, 0)
print(pd.crosstab(df['event_type'], df['purchase']))

purchase           0       1
event_type                  
cart         3028930       0
purchase           0  916939
view        63556110       0


In [30]:
df['user_session'] = df['user_session'].astype('category').cat.codes.astype('category')
df['user_id'] = df['user_id'].astype('category').cat.codes.astype('category')
df['product_id'] = df['product_id'].astype('category').cat.codes.astype('category')
df.tail()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,purchase
67501974,2019-11-30 23:59:58+00:00,view,4628,NaN,NaN,277.74,2281153,13764788,0
67501975,2019-11-30 23:59:58+00:00,view,37808,apparel.shoes,baden,62.81,227720,13769867,0
67501976,2019-11-30 23:59:59+00:00,view,417,electronics.smartphone,samsung,167.03,3613608,13769709,0
67501977,2019-11-30 23:59:59+00:00,view,9047,appliances.kitchen.refrigerators,samsung,566.27,460280,13767080,0
67501978,2019-11-30 23:59:59+00:00,view,23937,electronics.smartphone,apple,1312.52,3696116,13771164,0


In [31]:
df_targets = df.drop_duplicates(subset=['event_type', 'product_id', 'user_id', 'user_session'])
print(df_targets.shape)
print(df_targets.columns)
print(df_targets.dtypes)
df_targets.tail()

(44861467, 9)
Index(['event_time', 'event_type', 'product_id', 'category_code', 'brand',
       'price', 'user_id', 'user_session', 'purchase'],
      dtype='object')
event_time       datetime64[ns, UTC]
event_type                  category
product_id                  category
category_code               category
brand                       category
price                        float64
user_id                     category
user_session                category
purchase                       int32
dtype: object


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,purchase
67501970,2019-11-30 23:59:55+00:00,view,5159,furniture.kitchen.table,aero,1106.85,3617366,13772890,0
67501971,2019-11-30 23:59:57+00:00,view,15859,electronics.telephone,prestigio,8.37,3696100,13774792,0
67501972,2019-11-30 23:59:57+00:00,view,24382,construction.tools.drill,bosch,266.16,2546436,13769777,0
67501973,2019-11-30 23:59:57+00:00,view,857,NaN,NaN,29.60,1719499,13771312,0
67501978,2019-11-30 23:59:59+00:00,view,23937,electronics.smartphone,apple,1312.52,3696116,13771164,0


In [32]:
print(df_targets["category_code"].sample(frac=0.01).nunique())
df_targets["category_code"].value_counts()

127


electronics.smartphone          10672788
apparel.shoes                    1480471
electronics.clocks               1426793
computers.notebook               1358579
electronics.video.tv             1285448
                                  ...   
apparel.shorts                       561
construction.tools.screw             121
appliances.kitchen.fryer              68
country_yard.furniture.bench           2
apparel.jacket                         1
Name: category_code, Length: 129, dtype: int64

In [33]:
df_targets["category_code_level1"] = df_targets["category_code"].str.split(".",expand=True)[0].astype('category')

print(df_targets["category_code_level1"].nunique())
df_targets["category_code_level1"].value_counts()

13


electronics     15526294
appliances       5072242
computers        2644624
apparel          2449721
furniture        1473797
auto              756348
construction      678286
kids              582894
accessories       310619
sport             163900
medicine           12346
country_yard       11217
stationery          8432
Name: category_code_level1, dtype: int64

In [34]:
df_targets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44861467 entries, 0 to 67501978
Data columns (total 10 columns):
 #   Column                Dtype              
---  ------                -----              
 0   event_time            datetime64[ns, UTC]
 1   event_type            category           
 2   product_id            category           
 3   category_code         category           
 4   brand                 category           
 5   price                 float64            
 6   user_id               category           
 7   user_session          category           
 8   purchase              int32              
 9   category_code_level1  category           
dtypes: category(7), datetime64[ns, UTC](1), float64(1), int32(1)
memory usage: 2.8 GB


In [36]:
df_targets.to_csv("input/ecommerce-behavior/2019-Nov_Prep.csv", na_rep='NaN', index=False)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
categorical_dtypes = {
    'event_type':'category', 'product_id':'category',
    'category_code':'category', 'brand':'category', 
    'user_id':'category', 'user_session':'category'
}

In [3]:
df = pd.read_csv('input/ecommerce-behavior/2019-Nov_Prep.csv', dtype=categorical_dtypes)

In [5]:
df['event_time'] = pd.to_datetime(df['event_time'],infer_datetime_format=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44861467 entries, 0 to 44861466
Data columns (total 10 columns):
 #   Column                Dtype              
---  ------                -----              
 0   event_time            datetime64[ns, UTC]
 1   event_type            category           
 2   product_id            category           
 3   category_code         category           
 4   brand                 category           
 5   price                 float64            
 6   user_id               category           
 7   user_session          category           
 8   purchase              int64              
 9   category_code_level1  object             
dtypes: category(6), datetime64[ns, UTC](1), float64(1), int64(1), object(1)
memory usage: 3.0+ GB
